In [1]:
%%local
import os
import json
from IPython import get_ipython

username = os.environ['RENKU_USERNAME']

configuration = dict(
    name = f"{username}-final-project",
    executorMemory = "4G",
    executorCores = 4,
    numExecutors = 10,
    conf = {
        # "spark.pyspark.python": "/opt/anaconda3/bin/python3", # Use python3
        "spark.jars.repositories": "https://repos.spark-packages.org",
        "spark.jars.packages": "graphframes:graphframes:0.8.2-spark2.4-s_2.11"
    }
)


get_ipython().run_cell_magic('configure', line="-f", 
                             cell=json.dumps(configuration))

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2634,application_1680948035106_2467,pyspark,idle,Link,Link,None,
2641,application_1680948035106_2470,pyspark,idle,Link,Link,None,
2644,application_1680948035106_2471,pyspark,idle,Link,Link,None,
2650,application_1680948035106_2477,pyspark,idle,Link,Link,None,
2660,application_1680948035106_2485,pyspark,idle,Link,Link,None,
2663,application_1680948035106_2487,pyspark,idle,Link,Link,None,
2667,application_1680948035106_2490,pyspark,idle,Link,Link,None,
2668,application_1680948035106_2492,pyspark,idle,Link,Link,None,
2669,application_1680948035106_2493,pyspark,busy,Link,Link,None,
2671,application_1680948035106_2495,pyspark,idle,Link,Link,None,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2673,application_1680948035106_2497,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# load data from hdfs under /data/sbb/csv/timetable/*/2019/05/07, where * can be calendar, routes, stop_times and trips
file_path_timetable = "/data/sbb/part_orc/timetables/"

stop_times = spark.read.orc(file_path_timetable + "stop_times")

trips = spark.read.orc(file_path_timetable + "trips")

routes = spark.read.orc(file_path_timetable + "routes")

calendar = spark.read.orc(file_path_timetable + "calendar")

stops = spark.read.orc(file_path_timetable + "stops")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
actual = spark.read.load("/data/sbb/part_orc/istdaten",
                     format="orc", sep=";", inferSchema="true", header="true")

actual = actual.withColumnRenamed("betriebstag", "date_of_trip")\
                .withColumnRenamed("fahrt_bezeichner", "trip_id")\
                .withColumnRenamed("betreiber_id", "operator_id")\
                .withColumnRenamed("betreiber_abk", "operator_abk")\
                .withColumnRenamed("betreiber_name", "operator_name")\
                .withColumnRenamed("produkt_id", "transport_type")\
                .withColumnRenamed("linien_id", "train_number(train)")\
                .withColumnRenamed("linien_text", "service type(train)")\
                .withColumnRenamed("umlauf_id", "circulation_id")\
                .withColumnRenamed("verkehrsmittel_text", "means_of_transport_text")\
                .withColumnRenamed("zusatzfahrt_tf", "if_additional")\
                .withColumnRenamed("faellt_aus_tf", "if_failed")\
                .withColumnRenamed("bpuic", "stop_id")\
                .withColumnRenamed("haltestellen_name", "stop_name")\
                .withColumnRenamed("ankunftszeit", "arrival_time")\
                .withColumnRenamed("an_prognose", "actual_arrival_time")\
                .withColumnRenamed("abfahrtszeit", "departure_time")\
                .withColumnRenamed("ab_prognose", "actual_departure_time")\
                .withColumnRenamed("durchfahrt_tf", "not_stop")
actual.printSchema()
actual.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_of_trip: string (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- operator_id: string (nullable = true)
 |-- operator_abk: string (nullable = true)
 |-- operator_name: string (nullable = true)
 |-- transport_type: string (nullable = true)
 |-- train_number(train): string (nullable = true)
 |-- service type(train): string (nullable = true)
 |-- circulation_id: string (nullable = true)
 |-- means_of_transport_text: string (nullable = true)
 |-- if_additional: string (nullable = true)
 |-- if_failed: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- actual_arrival_time: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- actual_departure_time: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- not_stop: string (nullable = true)
 |-- year: integer (nullable = 